## Intro RL I - P4: Gym et FrozenLake

### 1. La librairie Gym

<img src="./openai.png" width="200" height="200" align="center"/>

Gym est une librairie Python développée et maintenue par OpenAI. La librairie contient une collection d'environnements prêts à être utilisés pour tester nos agents de RL. Elle contient des environnements avec des modèles précis (pour le model-based) et d'autres sans modèles (pour le model-free).

#### <b>Exemples d'environnements</b>

<img src="./gym_atari.png" width="400" height="200" align="center"/>

http://gym.openai.com/envs/

### 2. L'environnement FrozenLake

FrozenLake est un environnement de Gym de type gridworld. L'agent possède donc 4 actions : gauche (0), bas (1), droite (2), ou haut (3). Toutefois, les actions ne sont cette fois ci pas déterministes. Il y a 16 states (du state 0 au state 15) : un state = une case. 

<img src="./frozenlake_env.png" width="450" height="250" align="center"/>

<img src="./frozenlake_actions.png" width="200" height="250" align="center"/>
<center><em><span style="color:gray">La flèche en rouge montre l'action choisie par l'agent.</span></em></center>

In [3]:
import gym
env = gym.make('FrozenLake-v0')

In [4]:
env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

### 3. Résolution avec Policy Iteration

In [5]:
import numpy as np
import gym

In [6]:
env = gym.make('FrozenLake-v0')

In [7]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

In [8]:
def evaluate_policy(env, pi, V, gamma, theta):    
    V_updated = np.copy(V)
    improved = True
    
    while True:        
        delta = 0
        for s in range(env.nS):
            V_new = 0
        
            for a in range(env.nA):
                prob_a = pi[s][a]
                q_s_a = compute_q_value_for_s_a(env, V_updated, s, a, gamma)
            
                V_new += prob_a * q_s_a
        
            delta = max(delta, np.abs(V_new - V_updated[s]))
            V_updated[s] = V_new

        if(delta < theta):
            break
    
    if(np.array_equal(V, V_updated)):
        improved = False
    
    return V_updated, improved

def improve_policy(env, pi, V, gamma):    
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])
    
        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)
        
        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]
    
    return pi

In [9]:
pi = np.ones([env.nS, env.nA]) * 0.25
V = np.zeros([env.nS, 1])

gamma = 0.99 #facteur de return
theta = 0.00001 #seuil de similitude requis pour stopper les updates

In [10]:
i = 0
while True:
    i+=1
    
    V, improved = evaluate_policy(env, pi, V, gamma, theta)
    pi = improve_policy(env, pi, V, gamma)
    
    if(improved == False):
        print("Terminé après " + str(i) + " itérations.")
        break

Terminé après 540 itérations.


In [11]:
pi

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

<img src="./frozenlake_a_star.png" width="550" height="250" align="center"/>

### 4. Résolution avec Value Iteration

In [12]:
import numpy as np
import gym

In [13]:
env = gym.make('FrozenLake-v0')

In [14]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

In [15]:
def improve_policy(env, pi, V, gamma):    
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])
    
        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)
        
        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]
    
    return pi

In [16]:
V = np.zeros([env.nS, 1])

gamma = 0.99 #facteur de remise du return
theta = 0.00001 #seuil de similitude requis pour stopper les updates

In [17]:
i=0
while True:
    i+=1
    delta = 0
    
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])
        
        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

        newV = np.max(q_s)
        delta = max(delta, np.abs(newV - V[s]))
        V[s] = newV
        
    if(delta < theta):
        print("Terminé après " + str(i) + " itérations.")
        break

Terminé après 180 itérations.


In [18]:
V

array([[0.54190689],
       [0.49864543],
       [0.47051126],
       [0.4566538 ],
       [0.55834521],
       [0.        ],
       [0.35826844],
       [0.        ],
       [0.59171253],
       [0.64301781],
       [0.61515335],
       [0.        ],
       [0.        ],
       [0.74167772],
       [0.86281587],
       [0.        ]])

In [19]:
pi = np.ones([env.nS, env.nA]) * 0.25
pi = improve_policy(env, pi, V, gamma)

In [20]:
pi

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])